In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from __future__ import annotations
import sys
import re
import json
import time
from datetime import datetime, timezone
from dataclasses import dataclass, asdict

import requests
from bs4 import BeautifulSoup

@dataclass
class Rate:
    pair: str
    base: str
    quote: str
    price: float
    source: str
    fetched_at_utc: str

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/124.0.0.0 Safari/537.36",
    "Accept-Language": "es-ES,es;q=0.9,en;q=0.8",
}

URLS = {
    "USD-PEN": "https://www.google.com/finance/quote/USD-PEN?hl=es",
    "USD-DOP": "https://www.google.com/finance/quote/USD-DOP?hl=es",
    "USD-CNY": "https://www.google.com/finance/quote/USD-CNY?hl=es",
}

PRICE_REGEX = re.compile(r"[-+]?\d{1,3}(?:[.,]\d{3})*(?:[.,]\d+)?")

def fetch_price(url: str) -> float:
    """Descarga la página y busca el primer precio visible típico de Google Finance.
    Devuelve un float (punto como separador decimal).
    """
    resp = requests.get(url, headers=HEADERS, timeout=20)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")

    # 1) Marcado habitual del precio actual en Google Finance
    #    <div class="YMlKec fxKbKc">7.123</div>
    node = soup.find("div", class_="YMlKec fxKbKc")
    if node and node.text:
        txt = node.text.strip()
        # Normalizar: quitar símbolos no numéricos típicos
        m = PRICE_REGEX.search(txt.replace("\xa0", " "))
        if m:
            return float(m.group(0).replace(".", "").replace(",", "."))

    # 2) Fallback: buscar la primera coincidencia de un número "parecido a precio"
    #    en todo el documento (menos robusto).
    body_text = soup.get_text(" ", strip=True)
    m2 = PRICE_REGEX.search(body_text)
    if m2:
        try:
            return float(m2.group(0).replace(".", "").replace(",", "."))
        except Exception:
            pass

    raise RuntimeError("No se pudo parsear el precio en la página. El HTML pudo haber cambiado.")

def main():
    results: list[Rate] = []
    now_utc = datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")
    for pair, url in URLS.items():
        base, quote = pair.split("-")
        try:
            price = fetch_price(url)
            results.append(Rate(
                pair=pair,
                base=base,
                quote=quote,
                price=price,
                source=url,
                fetched_at_utc=now_utc
            ))
            # Respetar un pequeño delay por cortesía
            time.sleep(1.0)
        except Exception as ex:
            print(f"[ERROR] {pair}: {ex}", file=sys.stderr)

    if not results:
        print("No se obtuvo ningún tipo de cambio.", file=sys.stderr)
        sys.exit(2)

    # Guardar CSV y JSON
    import csv, os
    out_csv = "fx_rates_today.csv"
    out_json = "fx_rates_today.json"
    with open(out_csv, "w", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        w.writerow(["pair", "base", "quote", "price", "source", "fetched_at_utc"])
        for r in results:
            w.writerow([r.pair, r.base, r.quote, f"{r.price:.6f}", r.source, r.fetched_at_utc])

    with open(out_json, "w", encoding="utf-8") as f:
        json.dump([asdict(r) for r in results], f, ensure_ascii=False, indent=2)

    # Imprimir resumen
    print("Tipos de cambio (hoy):")
    for r in results:
        print(f"  {r.pair}: {r.price:.6f} ({r.source})")
    print(f"\nGuardado en: {out_csv} y {out_json}")

if __name__ == "__main__":
    main()


Tipos de cambio (hoy):
  USD-PEN: 3.386000 (https://www.google.com/finance/quote/USD-PEN?hl=es)
  USD-DOP: 63.293000 (https://www.google.com/finance/quote/USD-DOP?hl=es)
  USD-CNY: 7.124000 (https://www.google.com/finance/quote/USD-CNY?hl=es)

Guardado en: fx_rates_today.csv y fx_rates_today.json
